In [28]:
from extract_feat import create_dataset, testset

In [29]:
out_pairs = testset("../../deft_corpus/data/deft_files/ntest3.pkl", "testdata.pkl")

885it [00:10, 82.42it/s] 


In [12]:
pos, neg = create_dataset("../../deft_corpus/data/deft_files/nall_train.pkl", "save_feat.pkl")

17819it [03:29, 84.99it/s] 
Error pairs: 3


In [13]:
pos, neg = create_dataset("../../deft_corpus/data/deft_files/ntest3.pkl", "save_feat_test.pkl")

885it [00:10, 82.04it/s] 
Error pairs: 0


In [3]:
import pickle
from random_forest import process
def load_pkl(file_path):
    with open(file_path, "rb") as f:
        data = pickle.load(f)
    return data

In [4]:
pos, neg = load_pkl("save_feat_test.pkl")

In [7]:
print(pos[0])

sentence id: 646
tag: T149
type: Term
has_frag: False

sentence id: 79
tag: T148
type: Definition
has_frag: False

relation: Direct-Defines


In [30]:
testset = load_pkl("testdata.pkl")
output_pkl = load_pkl("output.pkl")
target_dict = load_pkl("target_dict.pkl")

In [31]:
inv_dict = dict()
for k, v in target_dict.items():
    inv_dict[v] = k
output = [inv_dict[v] for v in output_pkl]

In [32]:
len(output)

497730

In [34]:
keys, values = [], []
for i, (pair, label) in enumerate(zip(testset, output)):
    if label == "0" or label == "None":
        continue
    feat1 = pair.feat1
    feat2 = pair.feat2
    # print(feat1)
    # print(feat2)
    # print(label)
    # break
    if feat1.tag.split("deft")[0] != feat2.tag.split("deft")[0]:
        continue
    keys.append(pair)
    values.append(label)

In [35]:
tag_dic = dict()

In [36]:
for i, (key, value) in enumerate(zip(keys, values)):
    if key.feat1.tag in tag_dic:
        tag_dic[key.feat1.tag].append((key.feat2.tag, value))
    else:
        tag_dic[key.feat1.tag] = [(key.feat2.tag, value)]

In [44]:
closest = dict()
for key, value in tag_dic.items():
    if len(value) == 1:
        closest[key] = value[0]
    else:
        v1 = int(key.split("deft")[-1][1:])
        vs = list(map(int, [a[0].split("deft")[-1][1:] for a in value]))
        import numpy as np
        vs = np.abs(np.array(v1) - np.array(vs))
        vs = vs.argmin()
        value = value[vs]
        closest[key] = value

In [55]:
def read_file(filename):
    olines, keys = [], []
    with open(filename) as f:
        lines = list(f.readlines())
        for idx, line in enumerate(lines):
            line = lines[idx].strip().split("\t")
            line = [tok.strip() for tok in line]
            if len(line) == 1:
                continue
            file_path = line[1]
            tag = line[5]
            key = file_path.split("/")[-1] + tag
            olines.append(line)
            keys.append(key)
    return olines, keys

def read_dir(path):
    olines, keys = [], []
    for file in sorted(os.listdir(path)):
        if file.endswith(".deft"):
            p = os.path.join(path, file)
            s1, s2 = read_file(p)
            olines.extend(s1)
            keys.extend(s2)
    return olines, keys

In [56]:
lines, keys = read_dir("../../deft_corpus/data/test_files/labeled/subtask_3")

In [64]:
from copy import deepcopy as cp
newlines = cp(lines)
for i in range(len(lines)):
    if keys[i] in closest:
        value = closest[keys[i]]
        newlines[i][-2] = value[0].split("deft")[-1]
        newlines[i][-1] = value[1]
    else:
        newlines[i][-2] = "-1"
        newlines[i][-1] = "0"

In [65]:
def output(lst, file):
    with open(file, "w") as f:
        lst = ["\t".join(l) + "\n" for l in lst]
        f.writelines(lst)
output(lines, "../input/ref/task_3.deft")
output(newlines, "../input/res/task_3.deft")

In [46]:
closest

.deftT207', 'Direct-Defines'),
 't1_biology_0_202.deftT331': ('t1_biology_0_202.deftT330', 'Direct-Defines'),
 't1_biology_0_606.deftT20': ('t1_biology_0_606.deftT18', 'Direct-Defines'),
 't1_biology_0_606.deftT42': ('t1_biology_0_606.deftT41', 'Direct-Defines'),
 't1_biology_0_606.deftT62': ('t1_biology_0_606.deftT60', 'Direct-Defines'),
 't1_biology_0_606.deftT81': ('t1_biology_0_606.deftT82', 'Direct-Defines'),
 't1_biology_0_606.deftT99': ('t1_biology_0_606.deftT97', 'Direct-Defines'),
 't1_biology_0_606.deftT98': ('t1_biology_0_606.deftT97', 'AKA'),
 't1_biology_0_606.deftT122': ('t1_biology_0_606.deftT121', 'Direct-Defines'),
 't1_biology_0_606.deftT124': ('t1_biology_0_606.deftT123', 'Direct-Defines'),
 't1_biology_0_606.deftT125': ('t1_biology_0_606.deftT126', 'Direct-Defines'),
 't1_biology_0_606.deftT128': ('t1_biology_0_606.deftT127', 'Direct-Defines'),
 't1_biology_0_606.deftT144': ('t1_biology_0_606.deftT143', 'Direct-Defines'),
 't1_biology_0_606.deftT146': ('t1_biology_0